<div align="center"> 
    <hr>
  <font size="6"><b>Dev | VBB TEST</b> </font><br>
    <font size="6"><b>Abt - Buy</b> </font>
   <hr>
</div>

In [1]:
!python --version

Python 3.9.1


In [1]:
import os
import sys
import pandas as pd
import networkx
from networkx import draw, Graph
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
from pyjedai.utils import (
    text_cleaning_method,
    print_clusters,
    print_blocks,
    print_candidate_pairs
)
from pyjedai.evaluation import Evaluation, write
from pyjedai.datamodel import Data
# d1 = pd.read_csv("./data/test/ccer/abt_100.csv", sep='|', engine='python', na_filter=False).astype(str)
# d2 = pd.read_csv("./data/test/ccer/buy_100.csv", sep='|', engine='python', na_filter=False).astype(str)
# gt = pd.read_csv("./data/test/ccer/gt_100.csv", sep='|', engine='python')

d1 = pd.read_csv("./data/ccer/D2/abt.csv", sep='|', engine='python', na_filter=False).astype(str)
d2 = pd.read_csv("./data/ccer/D2/buy.csv", sep='|', engine='python', na_filter=False).astype(str)
gt = pd.read_csv("./data/ccer/D2/gt.csv", sep='|', engine='python')

attr1 = d1.columns[1:].to_list()
attr2 = d2.columns[1:].to_list()


data = Data(dataset_1=d1,
            attributes_1=attr1,
            id_column_name_1='id',
            dataset_2=d2,
            attributes_2=attr2,
            id_column_name_2='id',
            ground_truth=gt)


ModuleNotFoundError: No module named 'pyjedai'

# Block Building

## FAISS

faiss.IndexIVFFlat is an implementation of an inverted file index with coarse quantization. This index is used to efficiently search for nearest neighbors of a query vector in a large dataset of vectors. Here's a brief explanation of the parameters used in this index:

- `d`: The dimension of the input vectors. Each input vector should have d elements.
- `nlist`: The number of cells (or clusters) to divide the dataset into. This determines the number of inverted lists in the index, and hence the maximum number of vectors that can be stored in the index. Larger values of nlist will generally result in more accurate search results, but may also increase the memory requirements of the index and the time required to perform searches.
- `quantizer`: An instance of a quantizer object that is used to compress the vectors before they are added to the index. The quantizer object should implement a train method that learns the quantization function from a set of input vectors, and a compress method that compresses a vector into an integer code. FAISS provides several built-in quantizer objects, such as IndexFlatL2 and IndexScalarQuantizer.
- `metric`: The distance metric used to measure the similarity between vectors. The metric should be one of the strings defined in the faiss.METRIC_TYPES list, such as "l2" for Euclidean distance or "ip" for inner product.
- `nprobe`: The number of cells (or clusters) to search during a query. This parameter determines the tradeoff between accuracy and speed during searches. Larger values of nprobe will generally result in more accurate search results, but may also increase the time required to perform searches.



In [3]:
from pyjedai.vector_based_blocking import EmbeddingsNNBlockBuilding

In [4]:
emb = EmbeddingsNNBlockBuilding(vectorizer='sminilm',
                                similarity_search='faiss')
blocks, g = emb.build_blocks(data, 
                          top_k=5,
                            similarity_distance='cosine',
                          load_embeddings_if_exist=False,
                          save_embeddings=False,
                          with_entity_matching=True)

Building blocks via Embeddings-NN Block Building [sminilm, faiss]


Embeddings-NN Block Building [sminilm, faiss]:   0%|          | 0/2152 [00:00<?, ?it/s]

['name', 'description', 'price']
['name', 'description', 'price']
Device selected:  cpu
Creating sentence embeddings...
NORMALIZED
NORMALIZED
NORMALIZED


In [45]:
# 1.
import torch

from sentence_transformers import SentenceTransformer
from scipy import spatial, stats

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SentenceTransformer('all-MiniLM-L12-v2', device=device)

e2 = 0
print(emb._entities_d2[e2])
encoded_2 = model.encode(emb._entities_d2[e2])

res = []
for e1 in range(0, len(emb._entities_d1)):
    encoded_1 = model.encode(emb._entities_d1[e1])
    sim_score = 1 - spatial.distance.cosine(encoded_1, encoded_2)
    res.append((e1, sim_score))

linksys etherfast ezxs88w ethernet switch - ezxs88w linksys etherfast 8-port 10/100 switch (new/workgroup)


In [46]:
# 2

top_k = 5
sorted_list = sorted(res, key=lambda x: x[1], reverse=True)
print("Brute-force: ", sorted_list[:5])
print("pyJedAI:     ",[(x,y) for x,y in zip(emb.neighbors[0][:top_k], emb.distances[0][:top_k])])

Brute-force:  [(1023, 0.7504152059555054), (1020, 0.6593941450119019), (1022, 0.6422648429870605), (134, 0.6380040049552917), (1021, 0.6196042895317078)]
pyJedAI:      [(1023, 0.7504153), (1020, 0.6593942), (1022, 0.64226484), (134, 0.63800395), (1021, 0.6196043)]


In [47]:
emb.evaluate(blocks, with_classification_report=True, with_stats=True)

***************************************************************************************************************************
                                         Μethod:  Embeddings-NN Block Building
***************************************************************************************************************************
Method name: Embeddings-NN Block Building
Parameters: 
	Vectorizer: sminilm
	Similarity-Search: faiss
	Top-K: 5
	Vector size: 384
Runtime: 156.5100 seconds
───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Performance:
	Precision:      9.38% 
	Recall:        93.77%
	F1-score:      17.05%
───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Classification report:
	True positives: 1009
	False positives: 9751
	True negatives: 1147958
	False negatives: 67
	Total comparisons: 10760
──────────────────────────────────────────────

{'Precision %': 9.37732342007435,
 'Recall %': 93.77323420074349,
 'F1 %': 17.049678945589726,
 'True Positives': 1009,
 'False Positives': 9751,
 'True Negatives': 1147958,
 'False Negatives': 67}

# Entity Clustering

In [20]:
from pyjedai.clustering import ConnectedComponentsClustering, UniqueMappingClustering

In [27]:
ccc = UniqueMappingClustering()
clusters = ccc.process(g, data, similarity_threshold=0.7)

In [28]:
ccc.evaluate(clusters, with_classification_report=True)

***************************************************************************************************************************
                                         Μethod:  Unique Mapping Clustering
***************************************************************************************************************************
Method name: Unique Mapping Clustering
Parameters: 
Runtime: 0.0396 seconds
───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Performance:
	Precision:     82.78% 
	Recall:        67.47%
	F1-score:      74.35%
───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Classification report:
	True positives: 726
	False positives: 151
	True negatives: 1157275
	False negatives: 350
	Total comparisons: 877
───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


{'Precision %': 82.78221208665907,
 'Recall %': 67.4721189591078,
 'F1 %': 74.34715821812597,
 'True Positives': 726,
 'False Positives': 151,
 'True Negatives': 1157275,
 'False Negatives': 350}